# Responsável por tratar os dados pertinente a cliente

## Importando bibliotecas

In [2]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [6]:
import pandas as pd
from src.utils import carregar_tabela_excel

## Importando dados

In [11]:
source = "../../data/baseTesteIXC.xlsx"
df = carregar_tabela_excel(source)
df

,id,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,idCidade,Endereco,Complemento,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha
0,1306,Abelanio Veríssimo Peixoto,radiosnettelecom@gmail.com,(45)991198914,703.800.214-71,Isento,1659,Vila São Sebastião,Próximo ao cacimbão,0,Centro,57830-000,NaN,NaN,abelanio@radios.com,peixoto
1,897,Aclecia da Silva,radiosnettelecom@gmail.com,(82)998127951,126.947.984-97,30234875,1659,Maria de Lurdes,Próximo ao mirante,S/N,Centro,57830-000,NaN,NaN,aclecia@radios.com,silva
2,1120,Adeilton da Mata,adeiltondamata@gmail.com,(82)981229218,023.013.974-40,40259595,1659,Ass. Nova Esperança,próximo a casa de farinha,S/N,Zona Rural,57830-000,NaN,NaN,adeilton@radios.com,mata
3,1524,Adeilton da Silva,radiosnettelecom@gmail.com,(82)982806750,516.291.434-72,3614522-0,1659,Gravatá,Lote do Sr. Del,S/N,Zona Rural,57830-000,NaN,NaN,adeiltonn@radios.com,silva
4,1427,Adeilton Teixeira de Lima,radiosnettelecom@gmail.com,(82)999347915,008.433.544-02,36145220,1659,João Lyra / Qd: A,NaN,N: 24,Conj. Raimundo Nonato Lopes,57830-000,NaN,NaN,adeiltont@radios.com,lima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,1530,Yasmim emanuele correia da Silva,radiosnettelecom@gmail.com,NaN,475.748.218-36,NaN,1659,Rua marechal Floriano Peixoto,próximo ao sindicato,255,Mundo novo,57830-000,NaN,NaN,yasmin@radios.com,correia
619,422,Zaquel da Costa Firmino,betania@gmail.com,NaN,117.410.574-77,NaN,1659,Ass. Eldorado dos Carajás,mãe da Betânia,s/n,Zona Rural,57830-000,NaN,NaN,zaquel,costa
620,1223,Zélia Maria da Silva Oliveira,radiosnettelecom@gmail.com,NaN,100.483.964-27,NaN,1659,platô 3/ Q: 10,NaN,N:37,Conj. Raimundo Nonato Lopes,57830-000,NaN,NaN,zelia@radios.com,silva
621,387,ZENILDA DE LIMA FREIRE,radiosnettelecom@gmail.com,NaN,044.304.014-11,NaN,1659,Assentamento Flor do Mundau,irmã do dimas,SN,Zona Rural,57830-000,NaN,NaN,zenilda@radios.com,silva
